In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
weatherpy_database_df = pd.read_csv("weather_data/WeatherPy_Database.csv")
weatherpy_database_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Cabo San Lucas,MX,22.89,-109.91,89.60,45,5,8.05,clear sky,0.00,0.0
1,1,Egvekinot,RU,66.32,-179.17,41.77,99,100,3.94,light rain,0.34,0.0
2,2,Castro,BR,-24.79,-50.01,61.43,76,81,8.95,broken clouds,0.00,0.0
3,3,Chauk,MM,20.88,94.82,81.73,76,100,11.77,overcast clouds,0.00,0.0
4,4,Arman,RU,59.70,150.17,49.80,87,100,2.75,light rain,0.78,0.0


In [3]:
weatherpy_database_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [4]:
# Ask the customer to add a minimum and maximum temperature value and raining, snowing options.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = input("Do you want it to be raining? (yes/no) ")
snowing = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
# Filter the dataset to find the cities that fit the criteria.

if (raining == 'yes') and (snowing == 'yes'):
    preferred_cities_df = weatherpy_database_df.loc[(weatherpy_database_df["Max Temp"] <= max_temp) & \
                                                    (weatherpy_database_df["Max Temp"] >= min_temp) & \
                                                    (weatherpy_database_df["Rain (inches)"] > 0.0) & \
                                                    (weatherpy_database_df["Snow (inches)"] > 0)]
elif (raining == 'yes') and (snowing == 'no'):
    preferred_cities_df = weatherpy_database_df.loc[(weatherpy_database_df["Max Temp"] <= max_temp) & \
                                                    (weatherpy_database_df["Max Temp"] >= min_temp) & \
                                                    (weatherpy_database_df["Rain (inches)"] > 0.0) & \
                                                    (weatherpy_database_df["Snow (inches)"] == 0)]
elif (raining == 'no') and (snowing == 'yes'): 
    preferred_cities_df = weatherpy_database_df.loc[(weatherpy_database_df["Max Temp"] <= max_temp) & \
                                                    (weatherpy_database_df["Max Temp"] >= min_temp) & \
                                                    (weatherpy_database_df["Rain (inches)"] == 0.0) & \
                                                    (weatherpy_database_df["Snow (inches)"] > 0)]
else: 
    preferred_cities_df = weatherpy_database_df.loc[(weatherpy_database_df["Max Temp"] <= max_temp) & \
                                                    (weatherpy_database_df["Max Temp"] >= min_temp) & \
                                                    (weatherpy_database_df["Rain (inches)"] == 0.0) & \
                                                    (weatherpy_database_df["Snow (inches)"] == 0)]    

vacation_cities_df = preferred_cities_df

vacation_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
2,2,Castro,BR,-24.79,-50.01,61.43,76,81,8.95,broken clouds,0.0,0.0
7,7,Vaini,TO,-21.20,-175.20,73.40,100,90,9.17,light rain,0.0,0.0
12,12,Ribeira Grande,PT,38.52,-28.70,69.73,85,24,2.04,few clouds,0.0,0.0
17,17,Bambous Virieux,MU,-20.34,57.76,71.60,88,40,10.29,shower rain,0.0,0.0
18,18,Tete,MZ,-16.16,33.59,65.01,70,0,5.06,clear sky,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
551,551,Shirokiy,RU,49.76,129.52,60.42,82,38,9.08,scattered clouds,0.0,0.0
553,553,Naron,ES,43.52,-8.15,66.00,74,2,1.01,clear sky,0.0,0.0
554,554,Lleida,ES,41.62,0.62,73.99,74,0,6.38,clear sky,0.0,0.0
556,556,Zlobin,HR,45.29,14.65,64.40,52,0,8.01,clear sky,0.0,0.0


In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = vacation_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Castro,BR,61.43,broken clouds,-24.79,-50.01,
7,Vaini,TO,73.40,light rain,-21.20,-175.20,
12,Ribeira Grande,PT,69.73,few clouds,38.52,-28.70,
17,Bambous Virieux,MU,71.60,shower rain,-20.34,57.76,
18,Tete,MZ,65.01,clear sky,-16.16,33.59,
19,Tsiroanomandidy,MG,59.07,clear sky,-18.77,46.03,
20,Carutapera,BR,74.70,overcast clouds,-1.20,-46.02,
23,Rikitea,PF,69.67,overcast clouds,-23.12,-134.97,
28,Dukat,RU,61.56,overcast clouds,62.55,155.55,
29,Ponta Do Sol,PT,68.00,clear sky,32.67,-17.10,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Castro,BR,61.43,broken clouds,-24.79,-50.01,CHACARA BAILLY
7,Vaini,TO,73.40,light rain,-21.20,-175.20,Keleti Beach Resort
12,Ribeira Grande,PT,69.73,few clouds,38.52,-28.70,Quinta da Meia Eira
17,Bambous Virieux,MU,71.60,shower rain,-20.34,57.76,Casa Tia Villa
18,Tete,MZ,65.01,clear sky,-16.16,33.59,Zambeze Paraíso Misterioso
...,...,...,...,...,...,...,...
551,Shirokiy,RU,60.42,scattered clouds,49.76,129.52,
553,Naron,ES,66.00,clear sky,43.52,-8.15,Marcial
554,Lleida,ES,73.99,clear sky,41.62,0.62,Hotel Ramon Berenguer IV
556,Zlobin,HR,64.40,clear sky,45.29,14.65,Sobe Zlobin


In [11]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# marker layer map with pop-up marker
# Get hotel name, city, country, and current weather description with the max temp

city = vacation_cities_df["City"]
country = vacation_cities_df["Country"]
max_temp = vacation_cities_df["Max Temp"]
current_description = vacation_cities_df["Current Description"]
locations = vacation_cities_df[["Lat", "Lng"]]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the marker_layer variable.
marker_layer = gmaps.marker_layer(locations)

# Add the marker layer.
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}<dd><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

fig = gmaps.figure(center=(31.0, 31.0), zoom_level=1)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

from ipywidgets.embed import embed_minimal_html
embed_minimal_html('export.html', views=[fig])

# Call the figure to plot the data.
fig



Figure(layout=FigureLayout(height='420px'))